In [2]:
'''
@Author = Sree Teja Simha G

This is an implementation for grapheme to phoneme conversion. The lead inspiration for this
idea came from autoencoders. A specialized autoencoder for sequence to sequence mapping using RNN's
internal representation to target output.

The following code is a tutorial code from Keras.io for english to french translation.
It has been modified to fit word to phonemes conversion.

Sequence to sequence example in Keras (character-level) - modified to do symbol to symbol.

This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

# Summary of the algorithm

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.

- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [3]:
batch_size = 64  # Batch size for training.
epochs = 80  # Number of epochs to train for.
latent_dim = 64  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'ph.txt'

In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [5]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = target_text
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in ['\t']+target_text.split(' ')+['\n']:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [6]:
target_characters

['\t',
 '\n',
 'AA',
 'AE',
 'AH',
 'AO',
 'AW',
 'AY',
 'B',
 'CH',
 'D',
 'DH',
 'EH',
 'ER',
 'EY',
 'F',
 'G',
 'HH',
 'IH',
 'IY',
 'JH',
 'K',
 'L',
 'M',
 'N',
 'NG',
 'OW',
 'OY',
 'P',
 'R',
 'S',
 'SH',
 'T',
 'TH',
 'UH',
 'UW',
 'V',
 'W',
 'Y',
 'Z',
 'ZH']

In [7]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 26
Number of unique output tokens: 41
Max sequence length for inputs: 18
Max sequence length for outputs: 38


In [8]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [9]:
encoder_input_data = np.zeros(
    shape=(len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32'
)
decoder_input_data = np.zeros(
    shape=(len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    shape=(len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [10]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(['\t'] + target_text.split(' ') + ['\n']):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [11]:
encoder_input_data.shape

(10000, 18, 26)

In [12]:
decoder_input_data.shape

(10000, 38, 41)

In [13]:
decoder_target_data.shape

(10000, 38, 41)

In [14]:
# Define an input sequence and process it.
lstm1_inp = Input(
    shape=(None, num_encoder_tokens)
)
lstm_encoder = LSTM(
    latent_dim,
    return_state=True
)

#lstm_encoder input = lstm1_inp = word tensor
lstm_outputs, h_, c_ = lstm_encoder(lstm1_inp)

# lstm_outputs are not needed - we just need an internal representation
internal_states = [h_, c_]

In [15]:
# Decoder will start with the internal state of first lstm and incrementally build up final result
lstm2_inputs = Input(
    shape=(None, num_decoder_tokens)
)

# This lstm will take previous lstm's generated state
decoder_lstm = LSTM(
    latent_dim, # internal representation size
    return_sequences=True, # Well, these are our phonetic sequences
    return_state=True # We need internal states of decoder - but wont use them - unless we are stacking other LSTMs
)

# Dense NN
decoder_dense = Dense(
    num_decoder_tokens,
    activation='softmax'
)

#decoder_lstm_inputs = lstm2_inputs = decoder_input_data = phonemes tensor
# we use encoder internal state and previously predicted phoneme to
# predict the next phoneme
decoder_dense_inputs, _, _ = decoder_lstm(
    lstm2_inputs, # input of this lstm = Final results - we are forcing this lstm to map internal states to outputs
    initial_state=internal_states # initial state = internal states output of first lstm [h_, c_]
)
# secoder_dense_inputs = decoder_lstm_outputs
decoder_dense_outputs = decoder_dense(decoder_dense_inputs)

In [16]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model(
    inputs=[lstm1_inp, lstm2_inputs], # input of lstm1 and lstm2 in a pair
    outputs=decoder_dense_outputs
)

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 26)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 41)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 64), (None,  23296       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 64), ( 27136       input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [52]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
'''
The input for model training will be 
> encoder_input_data = one-hot representations of words
> decoder_input_data = one-hot representations of phonemes

The output shall be the next phoneme based on previous words - characters and previous - phonemes
~ decoder_targer_data = decoder_input_data << 1
'''
model.fit(
    x=[encoder_input_data, decoder_input_data], #encoder input and decoder input as input for the model
    y=decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

Train on 8000 samples, validate on 2000 samples
Epoch 1/80
8000/8000 [==============================] - 21s 3ms/step - loss: 0.6042 - val_loss: 0.5708
Epoch 2/80
8000/8000 [==============================] - 11s 1ms/step - loss: 0.5531 - val_loss: 0.5494
Epoch 3/80
8000/8000 [==============================] - 12s 1ms/step - loss: 0.5155 - val_loss: 0.5197
Epoch 4/80
8000/8000 [==============================] - 12s 2ms/step - loss: 0.4752 - val_loss: 0.4888
Epoch 5/80
8000/8000 [==============================] - 12s 2ms/step - loss: 0.4373 - val_loss: 0.4769
Epoch 6/80
8000/8000 [==============================] - 12s 2ms/step - loss: 0.4076 - val_loss: 0.4495
Epoch 7/80
8000/8000 [==============================] - 12s 2ms/step - loss: 0.3832 - val_loss: 0.4351
Epoch 8/80
8000/8000 [==============================] - 12s 2ms/step - loss: 0.3596 - val_loss: 0.4171
Epoch 9/80
8000/8000 [==============================] - 12s 2ms/step - loss: 0.3394 - val_loss: 0.4283
Epoch 10/80
8000/8000 [==

In [56]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
'''
Now, we reassemble the model using trained variables.

encoder is a model that takes lstm1_inp and generates internal_states
'''
encoder_model = Model(
    inputs=lstm1_inp,
    outputs=internal_states
)

'''
Decoder has 2 inputs - the hypotheses/state and classification/output of encoder
Together, they make up decoder input.
'''
ds_input_h = Input(shape=(latent_dim,))
ds_input_c = Input(shape=(latent_dim,))

ds_inputs = [ds_input_h, ds_input_c]

# This lstm constructs internal rep from lstm2_inputs = phonemes vector
# and internal state of first lstm
decoder_dense_inputs, dh, dc = decoder_lstm(
    lstm2_inputs, initial_state=ds_inputs)
ds = [dh, dc] # internal rep

# This internal rep is passed through a Dense NN(the one we previously trained)
decoder_dense_outputs = decoder_dense(decoder_dense_inputs)

# Input to the deocder model are
# > lstm2_inputs = previous phonemes
# > H state of previous LSTM(1)
# > C state of previous LSTM(1)

# Outputs of this model are
# > Decoder dense layer's output
# > H state of this LSTM(2)
# > C state of this LSTM(2)
decoder_model = Model(
    inputs = [lstm2_inputs] + ds_inputs,
    outputs = [decoder_dense_outputs] + ds)

# Reverse-lookup tables
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items()
)
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items()
)


def decode_sequence(input_batch):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_batch)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += [sampled_char]

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_batch = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_batch)
    print('-----------------------------------------------------------------------------------------------')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence[:-1])

-----------------------------------------------------------------------------------------------
Input sentence: aback
Decoded sentence: ['AH', 'B', 'AE', 'K']
-----------------------------------------------------------------------------------------------
Input sentence: abandon
Decoded sentence: ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N']
-----------------------------------------------------------------------------------------------
Input sentence: abandoned
Decoded sentence: ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'D']
-----------------------------------------------------------------------------------------------
Input sentence: abandoning
Decoded sentence: ['AH', 'B', 'AE', 'N', 'D', 'IH', 'NG', 'IH', 'NG']
-----------------------------------------------------------------------------------------------
Input sentence: abandons
Decoded sentence: ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'Z']
-----------------------------------------------------------------------------------------------
Input se

-----------------------------------------------------------------------------------------------
Input sentence: aborts
Decoded sentence: ['AH', 'B', 'AO', 'R', 'T', 'S']
-----------------------------------------------------------------------------------------------
Input sentence: abound
Decoded sentence: ['AH', 'B', 'AW', 'N', 'D']
-----------------------------------------------------------------------------------------------
Input sentence: abounded
Decoded sentence: ['AH', 'B', 'AW', 'N', 'D', 'IH', 'D']
-----------------------------------------------------------------------------------------------
Input sentence: abounding
Decoded sentence: ['AH', 'B', 'AW', 'N', 'D', 'IH', 'NG']
-----------------------------------------------------------------------------------------------
Input sentence: abounds
Decoded sentence: ['AH', 'B', 'AW', 'N', 'D', 'Z']
-----------------------------------------------------------------------------------------------
Input sentence: about
Decoded sentence: 